In [8]:
# Importing important libraries
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from warnings import filterwarnings

In [9]:
filterwarnings('ignore')

In [10]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Sumit
[nltk_data]     Singh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Sumit
[nltk_data]     Singh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
# Loading dataset
dataset=pd.read_csv('amazon_reviews.csv')
dataset.head(5)

Unnamed: 0  reviewerName  overall  \
0           0           NaN      4.0   
1           1          0mie      5.0   
2           2           1K3      4.0   
3           3           1m2      5.0   
4           4  2&amp;1/2Men      5.0   

                                          reviewText  reviewTime  day_diff  \
0                                         No issues.  2014-07-23       138   
1  Purchased this for my device, it worked as adv...  2013-10-25       409   
2  it works as expected. I should have sprung for...  2012-12-23       715   
3  This think has worked out great.Had a diff. br...  2013-11-21       382   
4  Bought it with Retail Packaging, arrived legit...  2013-07-13       513   

   helpful_yes  helpful_no  total_vote  score_pos_neg_diff  \
0            0           0           0                   0   
1            0           0           0                   0   
2            0           0           0                   0   
3            0           0           0                   0   
4            0           0           0                   0   

   score_average_rating  wilson_lower_bound  
0                   0.0                 0.0  
1                   0.0                 0.0  
2                   0.0                 0.0  
3                   0.0                 0.0  
4                   0.0                 0.0

In [12]:
# Checking dataset information and checking for is that any missing value in dataset or not
print(f"Information of the dataset is : \n{dataset.info()}\n")
print(f"\nChecking for any missing value: \n{dataset.isna().sum()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            4915 non-null   int64  
 1   reviewerName          4914 non-null   object 
 2   overall               4915 non-null   float64
 3   reviewText            4914 non-null   object 
 4   reviewTime            4915 non-null   object 
 5   day_diff              4915 non-null   int64  
 6   helpful_yes           4915 non-null   int64  
 7   helpful_no            4915 non-null   int64  
 8   total_vote            4915 non-null   int64  
 9   score_pos_neg_diff    4915 non-null   int64  
 10  score_average_rating  4915 non-null   float64
 11  wilson_lower_bound    4915 non-null   float64
dtypes: float64(3), int64(6), object(3)
memory usage: 460.9+ KB
Information of the dataset is : 
None


Checking for any missing value: 
Unnamed: 0              0
reviewerName    

In [13]:
# Feature selection
new_data=dataset[['reviewText', 'overall']]

In [ ]:
# Drop rows with missing reviewText
new_data = new_data.dropna(subset=['reviewText'])

# Ensure all reviewText entries are strings
new_data['reviewText'] = new_data['reviewText'].astype(str)

# Convert to lowercase
new_data['reviewText'] = new_data['reviewText'].str.lower()

# Remove punctuation
new_data['reviewText'] = new_data['reviewText'].str.replace(r'[^\w\s]', '', regex=True)

# removing numbers
new_data['reviewText'] = new_data['reviewText'].str.replace(r'\d+', '', regex=True)

# Remove stopwords
sw = set(stopwords.words('english'))
new_data['reviewText'] = new_data['reviewText'].apply(lambda x: " ".join(word for word in x.split() if word not in sw))

# Tokenization
new_data["tokens"] = new_data["reviewText"].apply(lambda x: word_tokenize(x))

# Lemmatization
lemmatizer = WordNetLemmatizer()
new_data["reviewText"] = new_data["tokens"].apply(lambda tokens: " ".join([lemmatizer.lemmatize(word) for word in tokens]))


In [18]:
# Converting text into numerica
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(new_data['reviewText']).toarray()
print(tfidf_matrix.shape)

(4914, 8981)


In [19]:
X=tfidf_matrix
y=new_data['overall']

In [20]:
# Spliting dataset into train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
#making model
from sklearn.ensemble import RandomForestClassifier

model=RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [22]:
#model prediction
y_pred= model.predict(X_test)

In [23]:
# Model Evolution
from sklearn.metrics import classification_report
print(f"Model Evaluation is : \n{classification_report(y_test, y_pred)}")

Model Evaluation is : 
              precision    recall  f1-score   support

         1.0       0.33      0.02      0.04        44
         2.0       0.00      0.00      0.00        12
         3.0       0.00      0.00      0.00        30
         4.0       0.00      0.00      0.00       107
         5.0       0.81      1.00      0.89       790

    accuracy                           0.80       983
   macro avg       0.23      0.20      0.19       983
weighted avg       0.66      0.80      0.72       983



In [29]:
text = ['it works as expected. I should have sprung for the higher capacity. I think its made a bit cheesier than the earlier versions; the paint looks not as clean as before']

t2=['This is bad']

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in sw]
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(lemmatized)

processed_text = [preprocess_text(t) for t in t2]
text_matrix = tfidf.transform(processed_text).toarray()



result = model.predict(text_matrix)

if (result>=3):
    print('Positive review')

elif (result<3 & result>=2):
    print("Average review")

else:
    print('Negative review')


Positive review


In [32]:
# saving the model
import pickle

with open("sentiment_model.pkl", "wb") as file:
    pickle.dump((model, tfidf), file)
print("✅ Model is saved successfully.")


✅ Model is saved successfully.
